In [1]:
%matplotlib inline
import os
from tomato.audio.AudioAnalyzer import AudioAnalyzer
from tomato.symbolic.SymbTrAnalyzer import SymbTrAnalyzer
from tomato.joint.JointAnalyzer import JointAnalyzer
import pylab
pylab.rcParams['figure.figsize'] = (20, 8)


JointAnalyzer assumes the individual audio analysis and score analysis is applied earlier. 

In [2]:
# score filepaths
symbtr_name = 'ussak--sazsemaisi--aksaksemai----neyzen_aziz_dede'
txt_score_filepath = os.path.join(symbtr_name, symbtr_name + '.txt')
mu2_score_filepath = os.path.join(symbtr_name, symbtr_name + '.mu2')

# instantiate
audio_mbid = 'f970f1e0-0be9-4914-8302-709a0eac088e'
audio_filepath = os.path.join(symbtr_name, audio_mbid, audio_mbid + '.mp3')

# instantiate analyzer objects
scoreAnalyzer = SymbTrAnalyzer(verbose=True)
audioAnalyzer = AudioAnalyzer(verbose=True)
jointAnalyzer = JointAnalyzer(verbose=True)


First we compute the input score and audio features for joint analysis.

In [3]:
# score (meta)data analysis
score_data = scoreAnalyzer.analyze(txt_score_filepath, mu2_score_filepath)

# predominant melody extraction
audio_pitch = audioAnalyzer.extract_pitch(audio_filepath)

# NOTE: do not call pitch filter later as aligned_pitch_filter will be more effective

- Automatic phrase segmentation on the SymbTr-txt file: ussak--sazsemaisi--aksaksemai----neyzen_aziz_dede/ussak--sazsemaisi--aksaksemai----neyzen_aziz_dede.txt
- Extracting (meta)data from the SymbTr-txt file: ussak--sazsemaisi--aksaksemai----neyzen_aziz_dede/ussak--sazsemaisi--aksaksemai----neyzen_aziz_dede.txt
- Extracting metadata from the SymbTr-mu2 file: ussak--sazsemaisi--aksaksemai----neyzen_aziz_dede/ussak--sazsemaisi--aksaksemai----neyzen_aziz_dede.mu2
- Extracting predominant melody of ussak--sazsemaisi--aksaksemai----neyzen_aziz_dede/f970f1e0-0be9-4914-8302-709a0eac088e/f970f1e0-0be9-4914-8302-709a0eac088e.mp3


Next, you can use the single line call "analyze," which does all the available analysis simultaneously. You can then update the audio analysis using the joint analysis results.

In [ ]:
# joint analysis
joint_features, score_informed_audio_features = jointAnalyzer.analyze(
    txt_score_filepath, score_data, audio_filepath, audio_pitch)

# redo some steps in audio analysis
score_informed_audio_features = audioAnalyzer.update_analysis(score_informed_audio_features)


- Extracting score-informed tonic and tempo of ussak--sazsemaisi--aksaksemai----neyzen_aziz_dede/f970f1e0-0be9-4914-8302-709a0eac088e/f970f1e0-0be9-4914-8302-709a0eac088e.mp3
- Aligning audio recording ussak--sazsemaisi--aksaksemai----neyzen_aziz_dede/f970f1e0-0be9-4914-8302-709a0eac088e/f970f1e0-0be9-4914-8302-709a0eac088e.mp3 and music score ussak--sazsemaisi--aksaksemai----neyzen_aziz_dede/ussak--sazsemaisi--aksaksemai----neyzen_aziz_dede.txt.
- Filtering predominant melody of ussak--sazsemaisi--aksaksemai----neyzen_aziz_dede/f970f1e0-0be9-4914-8302-709a0eac088e/f970f1e0-0be9-4914-8302-709a0eac088e.mp3 after audio-score alignment.
- Computing the note models for ussak--sazsemaisi--aksaksemai----neyzen_aziz_dede/f970f1e0-0be9-4914-8302-709a0eac088e/f970f1e0-0be9-4914-8302-709a0eac088e.mp3
- Obtaining the melodic progression model of ussak--sazsemaisi--aksaksemai----neyzen_aziz_dede/f970f1e0-0be9-4914-8302-709a0eac088e/f970f1e0-0be9-4914-8302-709a0eac088e.mp3
- Computing pitch distrib

... or the individual calls are given  below.

In [ ]:
# score (meta)data analysis
score_features = scoreAnalyzer.analyze(
    txt_score_filepath, mu2_score_filepath, symbtr_name=symbtr_name)

# predominant melody extraction
pitch = audioAnalyzer.extract_pitch(audio_filepath)

# joint analysis
# score-informed tonic and tempo estimation
tonic, tempo = jointAnalyzer.extract_tonic_tempo(
    txt_score_filepath, score_data, audio_filepath, audio_pitch)

# section linking and note-level alignment
sections, notes, section_candidates = jointAnalyzer.align_audio_score(
    txt_score_filepath, score_data, audio_filepath, audio_pitch, tonic, tempo)

# aligned pitch filter
pitch_filtered, notes_filtered = jointAnalyzer.filter_pitch(audio_pitch, notes)

# aligned note model
note_models, pitch_distribution, aligned_tonic = jointAnalyzer.get_note_models(
    pitch_filtered, notes_filtered, tonic['symbol'])

# recompute the audio features using the filtered pitch and tonic
# pitch histograms
pitch_class_distribution = pitch_distribution.to_pcd()

# get the melodic progression model
melodic_progression = audioAnalyzer.get_melodic_progression(pitch_filtered)

# transposition (ahenk) identification
transposition = audioAnalyzer.identify_transposition(aligned_tonic, aligned_tonic['symbol'])


- Automatic phrase segmentation on the SymbTr-txt file: ussak--sazsemaisi--aksaksemai----neyzen_aziz_dede/ussak--sazsemaisi--aksaksemai----neyzen_aziz_dede.txt
